In [4]:
%matplotlib inline

import torch
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
import time
from torchvision.models import resnet50, ResNet50_Weights

In [5]:
#Select device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [6]:
device

device(type='cuda')

In [22]:
#Dataset location
TRAIN_DATA_PATH = "../Data/CATS_DOGS/train"
TEST_DATA_PATH = "../Data/CATS_DOGS/test"

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {
    'train': 
    torchvision.datasets.ImageFolder(TRAIN_DATA_PATH, data_transforms['train']),
    'validation': 
    torchvision.datasets.ImageFolder(TEST_DATA_PATH, data_transforms['validation'])
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=32,
                                shuffle=True,
                                num_workers=16),  # for Kaggle
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=32,
                                shuffle=False,
                                num_workers=16)  # for Kaggle
}

In [9]:
device = 'cpu'

In [18]:
def test(model, data_loader):
    #Sets the module in evaluation mode
    model.eval()
    test_loss = 0
    correct = 0
    #correct.to(device)
    
    #dont update dynamic computation graph
    with torch.no_grad():
        #for every example in test
        for data, target in data_loader:
            
            target = target.view(-1, 1).float()
            target.to(device)
            
            #evaluate the model
            output = model(data.to(device))
            
            #acumulate the loss
            test_loss += F.binary_cross_entropy(output, target.to(device)).item()
            
            pred_cls = output.round()
            correct += pred_cls.eq(target.view(-1, 1).to(device)).sum() 
        
    test_loss /= len(data_loader.dataset)
    test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(data_loader.dataset), 100. * correct / len(data_loader.dataset)))

In [19]:
def train(model, data_loader, test_on_every_epoch = False):
    
    for epoch in range(1, num_epochs+1):
    
        
        if test_on_every_epoch:
            test()
        
        start = time.time()
  
        for batch_idx, (data, target) in enumerate(data_loader):
        
            model.train()
        
            #forward pass
            out = model(data.to(device))
        
            #Use negative log likelihood loss.
            loss = criterion(out, target.view(-1, 1).float().to(device))
        
        
            #with this gradients are calculated
            loss.backward()
    
            #update gradients
            optimizer.step()
        
            #Set gradients to zero
            optimizer.zero_grad()

            #Display iteration statistics
            if batch_idx % log_interval == 0:
            
                #print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(data_loader.dataset),100. * batch_idx / len(data_loader), loss.item()))
    
        end = time.time()
        print('Time: {} '.format(end - start))
    

In [16]:
# Hyper parameters
num_epochs = 4
#batchsize = 32
#batchsize_test = 32
learning_rate = 0.01
momentum = 0.5
log_interval = 60

#Network model
model = resnet50(weights="IMAGENET1K_V1").to(device)
#Disable training for all parameters in pretrained model
for param in model.parameters():
    param.requires_grad = False   

#Change last fully conected layer to a one with 2 outputs. This layer is trainable.
model.fc = torch.nn.Sequential(
               torch.nn.Linear(2048, 128),
               torch.nn.ReLU(inplace=True),
               torch.nn.Linear(128, 2)).to(device)


#Stochastic gradient decent
#optimizer = optim.SGD(network.parameters(), lr=learning_rate,momentum=momentum)
#Binary cross entropy loss
#criterion = torch.nn.BCELoss()

optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()



#train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=TRANSFORM_IMG)
#train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batchsize, shuffle=True,  num_workers=16)

#test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=TRANSFORM_IMG)
#test_data_loader  = torch.utils.data.DataLoader(test_data, batch_size=batchsize_test, shuffle=True, num_workers=16) 

train_losses = []
train_counter = []
test_losses = []

In [35]:
it = iter(dataloaders['train'])
data, target = next(it)
data.shape

torch.Size([32, 3, 224, 224])

In [23]:
train(model, dataloaders['train'])

RuntimeError: 0D or 1D target tensor expected, multi-target not supported

### Resources

- https://www.kaggle.com/code/pmigdal/transfer-learning-with-resnet-50-in-pytorch